# Marc tools

## Import

In [ ]:
import os
import plotly
import importlib
import re
import h5py

import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
import marc_tools as mt
import plotly.express as px

from pathlib import Path
from IPython.display import clear_output

# Set up plotly
plotly.offline.init_notebook_mode()

pio.templates['custom'] = go.layout.Template(
    layout_paper_bgcolor='rgba(0,0,0,0)',
    layout_plot_bgcolor='rgba(0,0,0,0)',
    ) 

# Change background color if using VSC
if 'VSCODE_CWD' in os.environ:
    pio.templates.default = 'plotly+custom'

# Pandas decimal precision
pd.set_option("display.precision", 2)

In [ ]:
# Define folders
project_dir = Path(r'C:\Projects\1993_Pierce_GI_riser')
bflex_dir = project_dir / 'Bflex/Local/Top_300bar'

### BFLEX - Stress and curvature

In [1001]:
# Find curvature
for curv in bflex_dir.glob(r'*elcurvY.mpf'):
    df_curv = mt.bflex.mpf_reader(curv)

df_curv.columns = ['curv']

# Find mpf_files
df_mpf = pd.DataFrame()

for mpf in bflex_dir.glob(r'*nosl?.mpf'):
    
    df_temp = mt.bflex.mpf_reader(mpf)
    df_mpf = pd.concat([df_mpf, df_temp], axis=1)

# Manipulate df
df = pd.DataFrame()

for i in range(17, 33):
    df[f'P{i}'] = df_mpf.filter(regex=f'{i}').max(axis=1)

df_stress = pd.concat([df_curv, df], axis=1)

# Interpolate curvature
df = pd.DataFrame({'curv' : [i/100 for i in range(0, 21)]})

for col in df_stress.columns[1:]:
    df[col] = np.interp(df.curv, df_stress.curv, df_stress[col])

# df_new.columns = ['Curvature'] + [i*22.5 for i in range(0, 16)]
pd.set_option("display.precision", 2)
df_bflex = df

In [1002]:
# Plotting
fig_bf = go.Figure()

for inc in range(0, 21, 5):
    fig_bf.add_trace(go.Scatterpolar(r=df.iloc[inc, 1:],
                                  theta=[-90 + 22.5*i for i in range(0, 17, 1)],
                                  fill='toself',
                                  name=f'Curv {df.curv.loc[inc]}'
                                  ))

fig_bf.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 600]
        )),
    showlegend=True
)

fig_bf.update_layout(title='Stress BFLEX',
                  autosize=False,
                  font=dict(color='white'),
                  width=1000, height=800)

fig_bf.show()

### Marc

#### Stress

In [949]:
# Bad incs
bad_incs = [26, 28, 29, 38, 39, 40, 46, 47, 51, 57, 64, 65, 67]

# Read Marc files
marc_dir = Path(r'\\nas-ask-001\Projects\1993_Pierce_GIR_fatigue\Marc\jupyter_notebook')
marc_files = marc_dir.glob('*stresses_inner_ta_inc*[0-9].csv*')

# Read list with coordinates per node
df_list = pd.read_csv(marc_dir / 'angles.csv', delimiter='\t').iloc[:,1:]
df_list.drop('node_index', axis=1, inplace=True)

# Iterate over files and create dataframe
df = pd.DataFrame()

for mf in marc_files:
    # Read increment
    inc = int(re.findall(r'\d+', mf.stem)[0])

    if inc in bad_incs:
        continue
    
    # Read dataframe and add x, y and z coordinate
    df_read = pd.read_csv(mf, index_col=None).iloc[:,1:]

    if all(df_read['node'] == df_list['node_id']):
        df_read = pd.concat([df_read, df_list[['z', 'theta']]], axis=1)

    df = pd.concat([df, df_read], axis=0)

df.sort_values(['inc', 'z', 'theta'], inplace=True)


In [994]:
df_marc = pd.DataFrame()

for inc in range(0, 71, 1):
    print(f'{inc/70*100:.0f} %')
    if inc in bad_incs:
        continue

    dfn = df[df.inc==inc]

    # Combine groups where difference in z positions is < 10 - check sort on z
    dfn['diff_z'] = dfn.groupby(['inc'])['z'].diff().gt(10).cumsum()

    # Sort dataframe based on diff_z and theta to allow .diff to work
    dfn.sort_values(['diff_z', 'theta'], inplace=True)

    # Combine groups where difference in theta is < 3
    dfn['diff_th'] = dfn.groupby(['inc', 'diff_z'])['theta'].diff().gt(3).cumsum()

    # Sort by diff and take max for each group
    dfg = dfn.groupby(['diff_z', 'diff_th']).max()
    dfg.reset_index(inplace=True)
    dfg.drop(columns=['diff_z', 'diff_th'], inplace=True)

    # dfg = dfn.groupby(['inc', 'z', 'diff']).max()
    # dfg.reset_index(inplace=True)
    # dfg.drop(['diff'], axis=1, inplace=True)

    df_marc = pd.concat([df_marc, dfg], axis=0)
    clear_output(wait=True)

# Round z
df_marc['z'] = [int(i) for i in np.round(list(df_marc['z']))]
df_marc['inc'] = [int(i) for i in df_marc['inc']]
df_marc['node'] = [int(i) for i in df_marc['node']]

# Space saving
df_marc = df_marc.astype({'theta': 'float32', 'stress': 'float32'})
clear_output()

In [1012]:
# Plotting
fig_marc = go.Figure()
pos = 26
marc_pos = sorted(list(set(df_marc.z)))

for inc in range(10, 71, 10):
    
    # Skip bad increments
    while inc in bad_incs:
        inc += 1

    # Only plot relevant and duplicate frist row last to close loop
    df_plot = df_marc[(df_marc.inc==inc) & (df_marc.z==marc_pos[pos])]

    # Drop every n'th row (for plotting purposes)
    df_plot = df_plot.iloc[::3]
    df_plot = df_plot.append(df_plot.reset_index(drop=True).iloc[0]).reset_index(drop=True)

    # Plot polar plot
    fig_marc.add_trace(go.Scatterpolar(r=df_plot['stress'],
                                  theta=df_plot['theta'],
                                  fill='toself',
                                  name=f'Inc {inc}'
                                  ))

fig_marc.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 500]
        )),
    showlegend=True
)

fig_marc.update_layout(title=f'Stress Marc @ {marc_pos[pos]}',
                  autosize=False,
                  font=dict(color='white'),
                  width=1000, height=800)

fig_marc.show()

## CrossSection - Read files

In [997]:
case_name = r'nov_8inch_gir_3p_with_bs_g3_job1'
project_folder = Path(r'V:\1993_Pierce_GIR_fatigue\Marc\analysis')

runs = ['CS'] # Folder with runs
exclude_cases = [] # optional - cases to be excluded from post-processing

df_case = mt.crossection.df_cs(project_folder, runs, positions='start')

In [998]:
# Marc list
marc_list = sorted(list(set(df_marc.z)))

# Cross section list
cs_list = [int(k) for k in df_case[case_name].keys()]

# Find closest values
cs_dict = {}
for cs in cs_list:
    cs_dict[cs] = min(marc_list, key=lambda x: abs(x - cs))

# Add column
df_marc['curv'] = np.nan

# Reduce dataframe to points where CS data is available
for i, (cs_pos, marc_pos) in enumerate(cs_dict.items(), 1):
    print(f'{i/len(cs_dict)*100:.0f} %')
    for inc in range(0, 71, 1):
        df_marc.loc[(df_marc.z==marc_pos) & (df_marc.inc==inc), 'curv'] = df_case[case_name][str(cs_pos)]['K3_4'][inc]
    clear_output(wait=True)

# Remove rows where curv is na
df_marc = df_marc[df_marc.curv.notna()]
# clear_output()

100 %


### Calculate SCF

In [999]:
# Initialize data frame
df_scf = pd.DataFrame()
valer = []

# Dictionary translating degrees to bflex position name
bflex_name = {-90 + 22.5*i: f'P{17+i}' for i in range(0, 16, 1)}

for i, marc_pos in enumerate(sorted(list(set(df_marc.z))), 1):
    print(f'Pos {i} of {len(list(set(df_marc.z)))}')

    # Find dictionary with bflex vs marc angles
    theta_dict = {}
    dft = df_marc[(df_marc.z==marc_pos) & (df_marc.inc==0)]

    for bf_angle in [22.5*i for i in range(0, 16, 1)]:
        theta_dict[bf_angle] = min(dft.theta, key=lambda x: abs(x - bf_angle))

    # Loop through increments
    for inc in range(0, 71, 10):
        # Skip inrements with bad input
        # if inc in bad_incs:
        #     continue

        while inc in bad_incs:
            inc+=1

        # Create dataframe containing specific position and inc only
        dft = df_marc[(df_marc.z==marc_pos) & (df_marc.inc==inc)]

        # Find stress for correct curvature and angle
        df_temp = dft[dft.theta.isin(theta_dict.values())]

        # Find corresponding bflex stress
        curv = df_temp.curv.iloc[0]
        df_interp = df_bflex.iloc[(df_bflex.curv - curv).abs().argsort()[:2]]
        y1, y2 = df_interp.iloc[0,1:], df_interp.iloc[1,1:]

        # Interpolate marc stress
        linfit = scipy.interpolate.interp1d(df_interp.curv, np.vstack([y1, y2]), axis=0)

        try:
            df_temp['stress_bflex'] = linfit(curv)
        except ValueError:
            valer.append(marc_pos)
            continue

        # Calculate the SCF
        df_temp['scf'] = df_temp.stress / df_temp.stress_bflex

        df_scf = pd.concat([df_scf,
                            df_temp[['inc', 'z', 'theta', 'curv', 'stress',
                                     'stress_bflex', 'scf']]
                            .reset_index(drop=True)], axis=0)
        clear_output(wait=True)
print(f'ValueError at {sorted(list(set(valer)))} mm')

ValueError at [388, 718, 821, 1213, 1543, 1646, 2037, 2367, 2862, 3192, 3295, 3686, 4016, 4511, 4841] mm


In [1005]:
# Plotting
fig_scf = go.Figure()
positions = sorted(list(set(df_scf.z)))
pos = 22
for inc in range(0, 71, 10):

    while inc in bad_incs:
        inc += 1
    
    # Extrat specific positinos
    df_plot = df_scf[(df_scf.inc==inc) & (df_scf.z==positions[pos])]
    df_plot = df_plot.append(df_plot.iloc[0]).reset_index()

    # Find curvature
    curv = df_plot.curv.iloc[0]

    # Plot
    fig_scf.add_trace(go.Scatterpolar(r=df_plot['scf'],
                                  theta=df_plot['theta'],
                                  fill='toself',
                                  name=f'Curvature {curv:.2f} ({inc})'
                                  ))

fig_scf.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 2.]
        )),
    showlegend=True
)

fig_scf.update_layout(title=f'Stress correction factor @ {positions[pos]} mm',
                      autosize=False,
                      font=dict(color='white'),
                      width=1000, height=800)

fig_scf.show()

In [659]:
df_plot

,index,inc,z,theta,curv,stress,stress_bflex,scf
0,0,70.0,1426.0,-88.48,0.1,29.35,191.36,0.15
1,1,70.0,1426.0,-65.00,0.1,33.88,252.76,0.13
2,2,70.0,1426.0,-41.52,0.1,64.52,303.53,0.21
3,3,70.0,1426.0,-25.87,0.1,93.19,335.93,0.28
4,4,70.0,1426.0,-2.39,0.1,156.89,345.10,0.45
5,5,70.0,1426.0,21.09,0.1,230.04,358.52,0.64
6,6,70.0,1426.0,44.57,0.1,288.82,345.20,0.84
7,7,70.0,1426.0,68.05,0.1,315.62,307.18,1.03
8,8,70.0,1426.0,91.52,0.1,322.32,250.56,1.29
9,9,70.0,1426.0,115.00,0.1,340.11,307.45,1.11


In [655]:
df_plot.append(df_plot.iloc[0]).reset_index()


,index,inc,z,theta,curv,stress,stress_bflex,scf
0,0,70.0,1426.0,-88.48,0.1,29.35,191.36,0.15
1,1,70.0,1426.0,-65.00,0.1,33.88,252.76,0.13
2,2,70.0,1426.0,-41.52,0.1,64.52,303.53,0.21
3,3,70.0,1426.0,-25.87,0.1,93.19,335.93,0.28
4,4,70.0,1426.0,-2.39,0.1,156.89,345.10,0.45
5,5,70.0,1426.0,21.09,0.1,230.04,358.52,0.64
6,6,70.0,1426.0,44.57,0.1,288.82,345.20,0.84
7,7,70.0,1426.0,68.05,0.1,315.62,307.18,1.03
8,8,70.0,1426.0,91.52,0.1,322.32,250.56,1.29
9,9,70.0,1426.0,115.00,0.1,340.11,307.45,1.11


## Examples

In [ ]:
# Plotly 3d surface with color - Example
df_plot = df_marc.where(df_marc['inc']==60).dropna()

df_plot['x'] = 200.0 * np.sin(df_plot['theta'])
df_plot['y'] = 200.0 * np.cos(df_plot['theta'])

fig = go.Figure()

fig.add_trace(go.Surface(
    x=df_plot['x'].tolist()[0:100],
    y=df_plot['y'].tolist()[0:100],
    z=[df_plot['z'].tolist()[0:100]]*100,
    surfacecolor=df_plot['stress'].tolist()[0:10]
))
fig.update_layout(title='Stress', autosize=True,
                  width=1000, height=800)

fig.update_layout(scene=dict(xaxis=dict(range=[-200, 200]),
                             yaxis=dict(range=[-200, 200]),
                             zaxis=dict(range=[0, 4000])
))
fig.show()

In [ ]:
# Plotly express
fig = px.line_polar(df_marc, r='stress', theta='theta', line_close=True)
fig.update_traces(fill='toself')
fig.show()

## Read marc input

In [ ]:
mf = Path(r'\\nas-ask-001\Projects\1993_Pierce_GIR_fatigue\Marc\analysis\NOV_8inch_GIR_3p_with_bs_g3_job1.h5')

with h5py.File(mf, 'r') as f:
    print(f'Keys {f["Marc"]["Results"]["Element"]["Tensor"]["Cauchy Stress in Preferred Sys"].keys()}')
